In [10]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import urllib
import re

def extract_marque(url):
    
    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")

    links = soup.find_all('a')
    
    data_list=[]
    for link in links:
        if link.string and link.string.strip() =='Voir plus':
            data=link['href']
            if data not in data_list:
                data_list.append(data)
    return data_list


In [7]:
import json
from urllib.request import urlopen,Request
from bs4 import BeautifulSoup
import ssl
import urllib
import re

def extract_modele(url):
    
    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")

    links = soup.find_all('a',href=lambda href: href and href.startswith(url))
#     print(links)
    data_list=[]
    for link in links:
        data=link['href']
        if data not in data_list:
            data_list.append(data)

    return data_list[:-1]


In [13]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import urllib
import re

def extract_version(url):
    
    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")

    links = soup.find_all('a')
    
    data_list=[]
    for link in links:
        span = link.find('span')
        if span and span.contents and 'Voir fiche technique' in ''.join(str(content) for content in span.contents):
            data = link['href']
            if data not in data_list:
                data_list.append(data)
    return data_list

In [29]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import urllib
import re

def extract_info(url):
    
    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    
    
    price=soup.find('div',class_="color_primary text_bold price-block")
    match = re.search(r'\b(\d{1,3}(?:\s\d{3})*)\b', str(price))
    price = int(match.group(1).replace(' ','')) if match else None
                                    
    links = soup.find('ul',class_="appliances")
    
    data_dict={}
    data_list=[]
    
    if links:
        li_elements = links.find_all('li')
        for li in li_elements:
            data = li.get_text(strip=True)
            data_list.append(data)
    data_list.append("price :"+str(price))   
    
    for item in data_list:
        key, value = item.split(":", 1)
        data_dict[key.strip()] = value.strip() 
    
    #print(data_dict)
    return data_dict


In [37]:
from urllib.parse import urlparse

def parse_car_url(url):
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')

    marque = path_parts[4]
    model = path_parts[5]
    version = path_parts[6].split('.')[0]
    
    # Create the dictionary
    car_dict = {'marque': marque, 'model': model, 'version': version}

    return car_dict


In [40]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import urllib
import re

def extract_car(url):
    
    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    cars_list=[]
    
    marques=extract_marque(url)
    for marque in marques :
        models=extract_modele(marque)
        
        for model in models:
            versions=extract_version(model)
            
            for version in versions:
                car_info=parse_car_url(version)
                car=extract_info(version)
                car_info.update(car)
                car_info['car_url']=str(url)
                cars_list.append(car_info)
                
        print(marque)
        
    json_file_path = "new_cars.json"
    with open(json_file_path, "w", encoding='utf-8') as json_file:
        json.dump(cars_list, json_file, ensure_ascii=False, indent=4)

    print(f"The data has been successfully exported to {json_file_path}.")            
    return cars_list


In [41]:

# Example URL
url = "https://www.moteur.ma/fr/neuf/fiche-technique-prix"

# Parse the URL and get the dictionary
extract_car(url)


https://www.moteur.ma/fr/neuf/fiche-technique-prix/abarth/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/alfa-romeo/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/aston-martin/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/audi/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/bentley/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/bmw/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/byd/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/chery/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/citroen/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/dacia/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/dfsk/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/ds/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/ferrari/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/fiat/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/ford/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/gaz/
https://www.moteur.ma/fr/neuf/fiche-technique-prix/geely/
https://

[{'marque': 'abarth',
  'model': '595',
  'version': 'abarth-595-595-turismo-165ch-2360',
  'Année de sortie du modèle': '2020 - 2024',
  'Carburant': 'Essence',
  'Puissance fiscale': '8 cv',
  'Puissance réelle': '165 ch',
  'Boite de vitesses': 'Manuelle',
  'price': '242403',
  'car_url': 'https://www.moteur.ma/fr/neuf/fiche-technique-prix'},
 {'marque': 'abarth',
  'model': '595',
  'version': 'abarth-595-595-competizione-180-2361',
  'Année de sortie du modèle': '2020 - 2024',
  'Carburant': 'Essence',
  'Puissance fiscale': '8 cv',
  'Puissance réelle': '180 ch',
  'Boite de vitesses': 'Manuelle',
  'price': '271503',
  'car_url': 'https://www.moteur.ma/fr/neuf/fiche-technique-prix'},
 {'marque': 'abarth',
  'model': '595-cabriolet',
  'version': 'abarth-595-cabriolet-595-cabriolet-145ch-2362',
  'Année de sortie du modèle': '2020 - 2024',
  'Carburant': 'Essence',
  'Puissance fiscale': '8 cv',
  'Puissance réelle': '145 ch',
  'Boite de vitesses': 'Manuelle',
  'price': '24490

In [54]:
import json
import csv

# Assuming the JSON file is named 'old_cars.json'
json_file_path = 'new_cars.json'

# Read the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    cars_list = json.load(json_file)

# Specify the attributes to include in the CSV file
selected_attributes = ["marque", "model", "Puissance fiscale", "Carburant", "Année de sortie du modèle", "Boite de vitesses","price"]

# Specify the CSV file path
csv_file_path = "new_cars.csv"

# Write the header to the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=selected_attributes)
    csv_writer.writeheader()

    # Write each car's information to the CSV file
    for car_info in cars_list:
        selected_info = {attr: car_info.get(attr, "") for attr in selected_attributes}
        csv_writer.writerow(selected_info)

print(f"The data has been successfully exported to {csv_file_path}.")


The data has been successfully exported to new_cars.csv.


In [55]:
import pandas as pd

In [56]:
df=pd.read_csv('new_cars.csv')

In [57]:
df.head(25)

,marque,model,Puissance fiscale,Carburant,Année de sortie du modèle,Boite de vitesses,price
0,abarth,595,8 cv,Essence,2020 - 2024,Manuelle,242403
1,abarth,595,8 cv,Essence,2020 - 2024,Manuelle,271503
2,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,244900
3,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,269900
4,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,299900
5,alfa-romeo,4c,14 cv,Essence,2013 - 2024,Automatique,757500
6,alfa-romeo,4c,14 cv,Essence,2014 - 2024,Automatique,857500
7,alfa-romeo,giulia,9 cv,Diesel,2021 - 2024,Automatique,556900
8,alfa-romeo,giulia,9 cv,Diesel,2021 - 2024,Automatique,595000
9,alfa-romeo,giulia,9 cv,Essence,2022 - 2024,Automatique,665000


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   marque                     1116 non-null   object
 1   model                      1116 non-null   object
 2   Puissance fiscale          1116 non-null   object
 3   Carburant                  1116 non-null   object
 4   Année de sortie du modèle  1116 non-null   object
 5   Boite de vitesses          1108 non-null   object
 6   price                      1116 non-null   object
dtypes: object(7)
memory usage: 61.2+ KB


In [59]:
new_column_names = ["marque", "model", "Puissance fiscale", "Carburant", "Année", "Boite de vitesses","price"]
df.columns = new_column_names
df.head()

,marque,model,Puissance fiscale,Carburant,Année,Boite de vitesses,price
0,abarth,595,8 cv,Essence,2020 - 2024,Manuelle,242403
1,abarth,595,8 cv,Essence,2020 - 2024,Manuelle,271503
2,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,244900
3,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,269900
4,abarth,595-cabriolet,8 cv,Essence,2020 - 2024,Manuelle,299900


In [60]:
df['Année'] = df['Année'].str.extract(r'(\d{4})')
df.head(25)

,marque,model,Puissance fiscale,Carburant,Année,Boite de vitesses,price
0,abarth,595,8 cv,Essence,2020,Manuelle,242403
1,abarth,595,8 cv,Essence,2020,Manuelle,271503
2,abarth,595-cabriolet,8 cv,Essence,2020,Manuelle,244900
3,abarth,595-cabriolet,8 cv,Essence,2020,Manuelle,269900
4,abarth,595-cabriolet,8 cv,Essence,2020,Manuelle,299900
5,alfa-romeo,4c,14 cv,Essence,2013,Automatique,757500
6,alfa-romeo,4c,14 cv,Essence,2014,Automatique,857500
7,alfa-romeo,giulia,9 cv,Diesel,2021,Automatique,556900
8,alfa-romeo,giulia,9 cv,Diesel,2021,Automatique,595000
9,alfa-romeo,giulia,9 cv,Essence,2022,Automatique,665000


In [61]:
df['Puissance fiscale'] = df['Puissance fiscale'].str.extract('(\d+)')
df.head(25)

,marque,model,Puissance fiscale,Carburant,Année,Boite de vitesses,price
0,abarth,595,8,Essence,2020,Manuelle,242403
1,abarth,595,8,Essence,2020,Manuelle,271503
2,abarth,595-cabriolet,8,Essence,2020,Manuelle,244900
3,abarth,595-cabriolet,8,Essence,2020,Manuelle,269900
4,abarth,595-cabriolet,8,Essence,2020,Manuelle,299900
5,alfa-romeo,4c,14,Essence,2013,Automatique,757500
6,alfa-romeo,4c,14,Essence,2014,Automatique,857500
7,alfa-romeo,giulia,9,Diesel,2021,Automatique,556900
8,alfa-romeo,giulia,9,Diesel,2021,Automatique,595000
9,alfa-romeo,giulia,9,Essence,2022,Automatique,665000


In [63]:
df['etat'] = 1
new_order = ["marque", "model", "Puissance fiscale", "Carburant", "Année", "Boite de vitesses","etat","price"]
df = df[new_order]
df.head()

,marque,model,Puissance fiscale,Carburant,Année,Boite de vitesses,etat,price
0,abarth,595,8,Essence,2020,Manuelle,1,242403
1,abarth,595,8,Essence,2020,Manuelle,1,271503
2,abarth,595-cabriolet,8,Essence,2020,Manuelle,1,244900
3,abarth,595-cabriolet,8,Essence,2020,Manuelle,1,269900
4,abarth,595-cabriolet,8,Essence,2020,Manuelle,1,299900


In [64]:
df.to_csv('u_new_cars.csv', index=False, encoding='utf-8')